In [100]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import ipywidgets
import pycatch22 as catch22
from sklearn.manifold import TSNE
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

## Combined Instance Space for Hopf and Bistable Systems

Hypotheses:
- Qualitatively similar dynamics such as fixed point behaviour should overlap between the two systems

In [48]:
with open('hopf_FE_XY.pkl', 'rb') as f:
    FE_HOPF = pickle.load(f)
with open('bistable_FE.pkl', 'rb') as f:
    FE_BISTABLE = pickle.load(f)

In [99]:
FEATURES = np.vstack([FE_HOPF, FE_BISTABLE])
normalized_features = StandardScaler().fit_transform(FEATURES)
pca_reducer = PCA(n_components=2)
embedding = pca_reducer.fit_transform(normalized_features)
#embedding = TSNE(n_iter=5000).fit_transform(normalized_features)

In [92]:
def label_region_Hopf(row):
    if row['control param'] == -5:
        return 'FLHB' # CP is critical point/Hopf Bifurcation
    if row['control param'] == -0.5:
        return 'CLHB' 
    if row['control param'] == 0:
        return 'HB' # Fixed point upper branch
    if row['control param'] == 0.5:
        return 'CRHB' 
    if row['control param'] == 5:
        return 'FRHB' 

In [93]:
mus = [-5, -0.5, 0, 0.5, 5]
etas = np.linspace(0.01, 1, 10)
mus_plot = np.concatenate([np.tile(mus[i], 200) for i in range(5)])
etas_plot = np.tile(np.concatenate([np.tile(etas[i], 20) for i in range(10)]), 5)
df1 = pd.DataFrame(embedding[:1000, :])
df1['control param'] = mus_plot
df1['eta'] = etas_plot
df1['regime'] = df1.apply(lambda row: label_region_Hopf(row), axis=1)
df1['system'] = 'Hopf'

In [94]:
def label_region_bistable(row):
    if row['control param'] < -0.384:
        return 'Fixed Point (Lower Branch)' # fixed point lower branch
    if row['control param'] >= -0.384 and row['control param'] <=0.384:
        return 'Bistable' # Bistable region
    if row['control param'] > 0.384:
        return 'Fixed Point (Upper Branch)' # Fixed point upper branch

In [95]:
df2 = pd.DataFrame(embedding[1000:, :])
etas = np.linspace(0.01, 1, 50)
lambdas = np.linspace(-1, 1, 50)
lambdas_plot = np.array([np.repeat(lambdas[i], 50) for i in range(len(lambdas))]).ravel() # 
etas_plot = np.tile(etas, 50)
df2['control param'] = lambdas_plot
df2['eta'] = etas_plot
df2['regime'] = df2.apply(lambda row: label_region_bistable(row), axis=1)
df2['system'] = 'Bistable'

In [96]:
df_combined = pd.concat([df1, df2]) 
df_combined = df_combined.reset_index(drop=True) # reset indexes

In [97]:
fig = px.scatter(df_combined, x=0, y=1, color=df_combined['regime'],
    labels={'0': 'PC1', '1': 'PC2', 'color': r"$\lambda$"}, width=1200, height=800, custom_data=['control param', 'eta', 'regime', 'system'])
fig.update_traces(
    hovertemplate="<br>".join([
        "Control Param: %{customdata[0]:.3f}",
        "Eta: %{customdata[1]:.3f}",
        "Regime: %{customdata[2]}",
        "System: %{customdata[3]}"
    ])
)
fig.update_traces(marker=dict(size=3))
fig.update_layout(title_text='Catch22 Instance Space', title_x=0.5)
fig.show()